In [236]:
import pandas as pd
from docx import Document
import os
import numpy as np
import docx
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT 
from docx.enum.table import WD_ALIGN_VERTICAL, WD_ROW_HEIGHT_RULE
from docx.shared import Pt, Cm
from docx.oxml import OxmlElement
from docx.shared import Length

In [2]:
import rarfile
import os

def extract_rar(rar_path, extract_to):
    # Открываем RAR-архив
    with rarfile.RarFile(rar_path) as rf:
        # Проверяем, существует ли директория для распаковки, если нет — создаем
        if not os.path.exists(extract_to):
            os.makedirs(extract_to)
        
        # Извлекаем все файлы в указанную директорию
        rf.extractall(path=extract_to)

# Укажите путь к вашему RAR-архиву и директорию для извлечения
rar_path = 'doc.rar'
extract_to = 'doc'

extract_rar(rar_path, extract_to)


### 1. Таблица с данными о ведомостях

In [7]:
df_docs = pd.DataFrame(columns=['kvartal','videl','area','file_path'])
for file in [f for f in os.listdir('doc') if f != '.DS_Store']:
    doc = Document(f'doc/{file}')
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)

    list_full = []

    for i in full_text:
        list_full.append(i.split('   '))

    def flatten_list(nested_list):
        flat_list = []
        for item in nested_list:
            if isinstance(item, list):
                flat_list.extend(flatten_list(item)) 
            else:
                flat_list.append(item)
        return flat_list

    list_full = flatten_list(list_full)


    kvartal = (list_full[list_full.index('квартал №')+1].strip())
    videl = (list_full[list_full.index('  выдел №')+1].strip())
    area = float(list_full[list_full.index('  экспл. площадь')+1].replace('га','').strip().replace(',','.'))

    new_row = {'kvartal': kvartal, 'videl': videl, 'area': area,'file_path':f'doc/{file}'}
    df_docs.loc[len(df_docs)] = new_row

In [30]:
df_docs[(df_docs['kvartal']==173)&(df_docs['videl']==13)]

,kvartal,videl,area,file_path
96,173,13,0.857,doc/кв173 в13 перечет1.docx


In [51]:
df_docs = df_docs.rename(columns={'kvartal':'Квартал','videl':'Выдел, который пишем в ведомостях','area':'Площадь'})

### 2. Таблица откуда берем данные

In [22]:
df_from = pd.read_excel('dop.xlsx')
df_from['Площадь'] = df_from['Площадь'].round(3)

In [46]:
df_from[df_from['Площадь'].isin(df_docs['area'])]

,Квартал,"Выдел, который пишем в ведомостях",Площадь,Ель.8.деловая,Ель.8.дровяная,Ель.12.деловая,Ель.12.дровяная,Ель.16.деловая,Ель.16.дровяная,Ель.20.деловая,...,Береза.20.дровяная,Береза.24.деловая,Береза.24.полделовая,Береза.24.дровяная,Береза.28.деловая,Береза.28.полделовая,Береза.28.дровяная,Береза.32.деловая,Береза.32.полделовая,Береза.32.дровяная
0,294,16,0.200,0,17,12,17,11,10,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,294,15,1.289,3,144,86,128,67,58,35,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,294,14,0.276,1,37,14,20,15,12,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,294,9,2.735,3,170,95,143,114,98,104,...,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,236,44,2.152,3,170,111,166,168,143,143,...,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,79,2,0.213,0,8,12,18,12,10,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,75,14,0.624,1,49,7,11,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,75,8,0.916,2,116,68,103,79,67,31,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,75,2,0.085,0,3,3,4,2,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
### мерджим в одну таблицу

df_merge = df_from.merge(df_docs, how='left', on=['Квартал','Выдел, который пишем в ведомостях','Площадь'])

In [57]:
df_merge[df_merge.file_path.isna()]

,Квартал,"Выдел, который пишем в ведомостях",Площадь,Ель.8.деловая,Ель.8.дровяная,Ель.12.деловая,Ель.12.дровяная,Ель.16.деловая,Ель.16.дровяная,Ель.20.деловая,...,Береза.24.деловая,Береза.24.полделовая,Береза.24.дровяная,Береза.28.деловая,Береза.28.полделовая,Береза.28.дровяная,Береза.32.деловая,Береза.32.полделовая,Береза.32.дровяная,file_path
9,236,24,0.575,1,44,31,47,31,26,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,236,26,0.000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,175,10,0.150,0,17,7,10,7,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,175,10,1.294,2,116,28,42,29,25,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,175,10,0.000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,175,10,0.000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,173,20,0.633,2,83,60,90,53,46,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,173,13,0.856,2,94,52,79,105,89,37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,150,38,0.000,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,130,3,5.424,16,814,530,794,599,510,237,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3. Находим каждую ведомость по таблице и вписываем в нее значения

In [266]:
from docx import Document

row = df_merge[:1]


# Открываем существующий документ
doc = Document(row['file_path'].values[0])

# Находим первую таблицу в документе
table = doc.tables[0]


# Определяем диапазон для удаления значений
start_row = 7 
start_col = 1
end_col = 14   


values_8 = [row['Ель.8.деловая'].values[0], row['Ель.8.дровяная'].values[0], row['Береза.8.деловая'].values[0], row['Береза.8.полделовая'].values[0], row['Береза.8.дровяная'].values[0]]
if all(pd.isna(value) for value in values_8):
    sum_8 = np.nan  # Если все значения NaN
else:
    sum_8 = sum(value for value in values_8 if not pd.isna(value)) 

values_12 = [row['Ель.12.деловая'].values[0], row['Ель.12.дровяная'].values[0], row['Береза.12.деловая'].values[0], row['Береза.12.полделовая'].values[0], row['Береза.12.дровяная'].values[0]]
if all(pd.isna(value) for value in values_12):
    sum_12 = np.nan  # Если все значения NaN
else:
    sum_12 = sum(value for value in values_12 if not pd.isna(value)) 

values_16 = [row['Ель.16.деловая'].values[0], row['Ель.16.дровяная'].values[0], row['Береза.16.деловая'].values[0], row['Береза.16.полделовая'].values[0], row['Береза.16.дровяная'].values[0]]
if all(pd.isna(value) for value in values_16):
    sum_16 = np.nan  # Если все значения NaN
else:
    sum_16 = sum(value for value in values_16 if not pd.isna(value)) 

values_20 = [row['Ель.20.деловая'].values[0], row['Ель.20.дровяная'].values[0], row['Береза.20.деловая'].values[0], row['Береза.20.полделовая'].values[0], row['Береза.20.дровяная'].values[0]]
if all(pd.isna(value) for value in values_20):
    sum_20 = np.nan  # Если все значения NaN
else:
    sum_20 = sum(value for value in values_20 if not pd.isna(value)) 

values_24 = [row['Ель.24.деловая'].values[0], row['Ель.24.дровяная'].values[0], row['Береза.24.деловая'].values[0], row['Береза.24.полделовая'].values[0], row['Береза.24.дровяная'].values[0]]
if all(pd.isna(value) for value in values_24):
    sum_24 = np.nan  # Если все значения NaN
else:
    sum_24 = sum(value for value in values_24 if not pd.isna(value)) 

values_28 = [row['Ель.28.деловая'].values[0], row['Ель.28.дровяная'].values[0], row['Береза.28.деловая'].values[0], row['Береза.28.полделовая'].values[0], row['Береза.28.дровяная'].values[0]]
if all(pd.isna(value) for value in values_28):
    sum_28 = np.nan  # Если все значения NaN
else:
    sum_28 = sum(value for value in values_28 if not pd.isna(value)) 

values_32 = [row['Ель.32.деловая'].values[0], row['Ель.32.дровяная'].values[0], row['Береза.32.деловая'].values[0], row['Береза.32.полделовая'].values[0], row['Береза.32.дровяная'].values[0]]
if all(pd.isna(value) for value in values_32):
    sum_32 = np.nan  # Если все значения NaN
else:
    sum_32 = sum(value for value in values_32 if not pd.isna(value)) 


super_sum_el_del = row['Ель.8.деловая'].fillna(0).values[0]+row['Ель.12.деловая'].fillna(0).values[0]+row['Ель.16.деловая'].fillna(0).values[0]+row['Ель.20.деловая'].fillna(0).values[0]+row['Ель.24.деловая'].fillna(0).values[0]+row['Ель.28.деловая'].fillna(0).values[0]+row['Ель.32.деловая'].fillna(0).values[0]
super_sum_el_drov = row['Ель.8.дровяная'].fillna(0).values[0]+row['Ель.12.дровяная'].fillna(0).values[0]+row['Ель.16.дровяная'].fillna(0).values[0]+row['Ель.20.дровяная'].fillna(0).values[0]+row['Ель.24.дровяная'].fillna(0).values[0]+row['Ель.28.дровяная'].fillna(0).values[0]+row['Ель.32.дровяная'].fillna(0).values[0]
super_sum_ber_del = row['Береза.8.деловая'].fillna(0).values[0]+row['Береза.12.деловая'].fillna(0).values[0]+row['Береза.16.деловая'].fillna(0).values[0]+row['Береза.20.деловая'].fillna(0).values[0]+row['Береза.24.деловая'].fillna(0).values[0]+row['Береза.28.деловая'].fillna(0).values[0]+row['Береза.32.деловая'].fillna(0).values[0]
super_sum_ber_poldel = row['Береза.8.полделовая'].fillna(0).values[0]+row['Береза.12.полделовая'].fillna(0).values[0]+row['Береза.16.полделовая'].fillna(0).values[0]+row['Береза.20.полделовая'].fillna(0).values[0]+row['Береза.24.полделовая'].fillna(0).values[0]+row['Береза.28.полделовая'].fillna(0).values[0]+row['Береза.32.полделовая'].fillna(0).values[0]
super_sum_ber_drov = row['Береза.8.дровяная'].fillna(0).values[0]+row['Береза.12.дровяная'].fillna(0).values[0]+row['Береза.16.дровяная'].fillna(0).values[0]+row['Береза.20.дровяная'].fillna(0).values[0]+row['Береза.24.дровяная'].fillna(0).values[0]+row['Береза.28.дровяная'].fillna(0).values[0]+row['Береза.32.дровяная'].fillna(0).values[0]



sum_values = [sum_8, sum_16, sum_20, sum_24, sum_28, sum_32]
check_num = sum(value if not np.isnan(value) else 0 for value in sum_values)

new_data = [
    ["8", (row['Ель.8.деловая'].values[0]), "", "", (row['Ель.8.дровяная'].values[0]), (row['Береза.8.деловая'].values[0]), (row['Береза.8.полделовая'].values[0]), (row['Береза.8.дровяная'].values[0]),"","","","","","",sum_8],
    ["12", (row['Ель.12.деловая'].values[0]), "", "", (row['Ель.12.дровяная'].values[0]), (row['Береза.12.деловая'].values[0]), (row['Береза.12.полделовая'].values[0]), (row['Береза.12.дровяная'].values[0]),"","","","","","",sum_12],
    ["16", (row['Ель.16.деловая'].values[0]), "", "", (row['Ель.16.дровяная'].values[0]), (row['Береза.16.деловая'].values[0]), (row['Береза.16.полделовая'].values[0]), (row['Береза.16.дровяная'].values[0]),"","","","","","",sum_16],
    ["20", (row['Ель.20.деловая'].values[0]), "", "", (row['Ель.20.дровяная'].values[0]), (row['Береза.20.деловая'].values[0]), (row['Береза.20.полделовая'].values[0]), (row['Береза.20.дровяная'].values[0]),"","","","","","",sum_20],
    ["24", (row['Ель.24.деловая'].values[0]), "", "", (row['Ель.24.дровяная'].values[0]), (row['Береза.24.деловая'].values[0]), (row['Береза.24.полделовая'].values[0]), (row['Береза.24.дровяная'].values[0]),"","","","","","",sum_24],
    ["28", (row['Ель.28.деловая'].values[0]), "", "", (row['Ель.28.дровяная'].values[0]), (row['Береза.28.деловая'].values[0]), (row['Береза.28.полделовая'].values[0]), (row['Береза.28.дровяная'].values[0]),"","","","","","",sum_28],
    ["32", (row['Ель.32.деловая'].values[0]), "", "", (row['Ель.32.дровяная'].values[0]), (row['Береза.32.деловая'].values[0]), (row['Береза.32.полделовая'].values[0]), (row['Береза.32.дровяная'].values[0]),"","","","","","",sum_32],
    ["36","","","","","","","","","","","","","","",],
    ["40","","","","","","","","","","","","","","",],
    ["44","","","","","","","","","","","","","","",],
    ["48","","","","","","","","","","","","","","",],
    ["52","","","","","","","","","","","","","","",],
    ["56","","","","","","","","","","","","","","",],
    ["60","","","","","","","","","","","","","","",],
    ["64","","","","","","","","","","","","","","",],
    ["итого",super_sum_el_del,"", "",super_sum_el_drov,super_sum_ber_del,super_sum_ber_poldel,super_sum_ber_drov,"","","","","","",check_num]
]

for i in range(start_row, len(table.rows)):  # Проходим от 8-й строки до последней
    for j in range(start_col, end_col + 1):  # Проходим от 2-го до 15-го столбца
        cell = table.cell(i, j)
        cell.text = ""  # Очищаем ячейку

# Количество пустых строк перед заполнением
empty_rows_count = start_row

# Заполнение данных в таблице
for i, row in enumerate(new_data):
    for j, value in enumerate(row):
        # Проверяем на пустые строки и nan
        cell = table.cell(i + empty_rows_count, j)

        
        # Проверяем значение и заполняем ячейку
        if value == "" or (isinstance(value, float) and np.isnan(value)):
            cell.text = " "  # Или другое значение для пустой ячейки
        elif value == "итого":
            cell.text = value
        else:
            cell.text = str(int(value))  # Преобразуем значение в строку

        # Применяем шрифт "Times New Roman"
        for paragraph in cell.paragraphs:
            for run in paragraph.runs:
                run.font.name = 'Times New Roman'
                run.font.size = docx.shared.Pt(12)  # Устанавливаем размер шрифта (12 пунктов)
            paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
        # Устанавливаем вертикальное выравнивание по центру
        cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
        
        # Задаем высоту ячейки, чтобы вертикальное выравнивание работало
        cell.height = Cm(0.1) # Укажите нужную высоту ячейки


for row in table.rows[7:]:  # Начиная с 8-й строки (индекс 7)
    row.height = Pt(15)  # Установка высоты строки
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY



for paragraph in table.cell(22,0).paragraphs:
        paragraph.space_after = Pt(0)  # Устанавливаем расстояние после абзаца в 0
        paragraph.space_before = Pt(0)  # Устанавливаем расстояние перед абзацем в 0
        paragraph.paragraph_format.line_spacing = Pt(10)

for cell in table.rows[-1].cells:  # Используем .cells для получения ячеек
    cell.vertical_alignment = WD_ALIGN_VERTICAL.CENTER
    cell.height = Pt(25) # Укажите нужную высоту ячейки
table.rows[-1].height = Pt(25)


table.rows[1].height = Pt(16)
table.rows[1].height_rule = WD_ROW_HEIGHT_RULE.EXACTLY

table.rows[2].height = Pt(12)
table.rows[2].height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
table.rows[3].height = Pt(12)
table.rows[3].height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
table.rows[4].height = Pt(12)
table.rows[4].height_rule = WD_ROW_HEIGHT_RULE.EXACTLY

doc.save('updated_example.docx')

In [99]:
row

,Квартал,"Выдел, который пишем в ведомостях",Площадь,Ель.8.деловая,Ель.8.дровяная,Ель.12.деловая,Ель.12.дровяная,Ель.16.деловая,Ель.16.дровяная,Ель.20.деловая,...,Береза.24.деловая,Береза.24.полделовая,Береза.24.дровяная,Береза.28.деловая,Береза.28.полделовая,Береза.28.дровяная,Береза.32.деловая,Береза.32.полделовая,Береза.32.дровяная,file_path
0,294,16,0.2,0,17,12,17,11,10,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doc/кв294 в16 перечет1.docx


In [248]:
table.rows[2].cells[1].text

'деловой'

In [171]:
row = df_merge[:1]
new_data = [
    ["8", float(row['Ель.8.деловая'].values[0]), "", "", float(row['Ель.8.дровяная'].values[0]), float(row['Береза.8.деловая'].values[0]), float(row['Береза.8.полделовая'].values[0]), float(row['Береза.8.дровяная'].values[0]),"","","","","","",float(row['Ель.8.деловая'].fillna(0).values[0]+row['Ель.8.дровяная'].fillna(0).values[0]+row['Береза.8.деловая'].fillna(0).values[0]+row['Береза.8.полделовая'].fillna(0).values[0]+row['Береза.8.дровяная'].fillna(0).values[0])],
    ["12", float(row['Ель.12.деловая'].values[0]), "", "", float(row['Ель.12.дровяная'].values[0]), float(row['Береза.12.деловая'].values[0]), float(row['Береза.12.полделовая'].values[0]), float(row['Береза.12.дровяная'].values[0]),"","","","","","",float(row['Ель.12.деловая'].fillna(0).values[0]+row['Ель.12.дровяная'].fillna(0).values[0]+row['Береза.12.деловая'].fillna(0).values[0]+row['Береза.12.полделовая'].fillna(0).values[0]+row['Береза.12.дровяная'].fillna(0).values[0])],
    ["16", float(row['Ель.16.деловая'].values[0]), "", "", float(row['Ель.16.дровяная'].values[0]), float(row['Береза.16.деловая'].values[0]), float(row['Береза.16.полделовая'].values[0]), float(row['Береза.16.дровяная'].values[0]),"","","","","","",float(row['Ель.16.деловая'].fillna(0).values[0]+row['Ель.16.дровяная'].fillna(0).values[0]+row['Береза.16.деловая'].fillna(0).values[0]+row['Береза.16.полделовая'].fillna(0).values[0]+row['Береза.16.дровяная'].fillna(0).values[0])],
    ["20", float(row['Ель.20.деловая'].values[0]), "", "", float(row['Ель.20.дровяная'].values[0]), float(row['Береза.20.деловая'].values[0]), float(row['Береза.20.полделовая'].values[0]), float(row['Береза.20.дровяная'].values[0]),"","","","","","",float(row['Ель.20.деловая'].fillna(0).values[0]+row['Ель.20.дровяная'].fillna(0).values[0]+row['Береза.20.деловая'].fillna(0).values[0]+row['Береза.20.полделовая'].fillna(0).values[0]+row['Береза.20.дровяная'].fillna(0).values[0])],
    ["24", float(row['Ель.24.деловая'].values[0]), "", "", float(row['Ель.24.дровяная'].values[0]), float(row['Береза.24.деловая'].values[0]), float(row['Береза.24.полделовая'].values[0]), float(row['Береза.24.дровяная'].values[0]),"","","","","","",float(row['Ель.24.деловая'].fillna(0).values[0]+row['Ель.24.дровяная'].fillna(0).values[0]+row['Береза.24.деловая'].fillna(0).values[0]+row['Береза.24.полделовая'].fillna(0).values[0]+row['Береза.24.дровяная'].fillna(0).values[0])],
    ["28", float(row['Ель.28.деловая'].values[0]), "", "", float(row['Ель.28.дровяная'].values[0]), float(row['Береза.28.деловая'].values[0]), float(row['Береза.28.полделовая'].values[0]), float(row['Береза.28.дровяная'].values[0]),"","","","","","",float(row['Ель.28.деловая'].fillna(0).values[0]+row['Ель.28.дровяная'].fillna(0).values[0]+row['Береза.28.деловая'].fillna(0).values[0]+row['Береза.28.полделовая'].fillna(0).values[0]+row['Береза.28.дровяная'].fillna(0).values[0])],
    ["32", float(row['Ель.32.деловая'].values[0]), "", "", float(row['Ель.32.дровяная'].values[0]), float(row['Береза.32.деловая'].values[0]), float(row['Береза.32.полделовая'].values[0]), float(row['Береза.32.дровяная'].values[0]),"","","","","","",float(row['Ель.32.деловая'].fillna(0).values[0]+row['Ель.32.дровяная'].fillna(0).values[0]+row['Береза.32.деловая'].fillna(0).values[0]+row['Береза.32.полделовая'].fillna(0).values[0]+row['Береза.32.дровяная'].fillna(0).values[0])]
]
new_data

[['8', 0.0, '', '', 17.0, 1.0, 0.0, 14.0, '', '', '', '', '', '', 32.0],
 ['12', 12.0, '', '', 17.0, 7.0, 0.0, 12.0, '', '', '', '', '', '', 48.0],
 ['16', 11.0, '', '', 10.0, 4.0, 0.0, 10.0, '', '', '', '', '', '', 35.0],
 ['20', 11.0, '', '', 8.0, nan, nan, nan, '', '', '', '', '', '', 19.0],
 ['24', 0.0, '', '', 0.0, nan, nan, nan, '', '', '', '', '', '', 0.0],
 ['28', nan, '', '', nan, nan, nan, nan, '', '', '', '', '', '', 0.0],
 ['32', nan, '', '', nan, nan, nan, nan, '', '', '', '', '', '', 0.0]]

SyntaxError: unmatched ')' (3920986696.py, line 1)

In [158]:
new_data

[['8', 0, '', '', 17, 1, 0.0, 14.0, '', '', '', '', '', '', 32.0],
 ['12', 12, '', '', 17, 7.0, 0.0, 12.0, '', '', '', '', '', '', 48.0],
 ['16', 11, '', '', 10, 4.0, 0.0, 10.0, '', '', '', '', '', '', 35.0],
 ['20', 11, '', '', 8, nan, nan, nan, '', '', '', '', '', '', nan],
 ['24', 0, '', '', 0, nan, nan, nan, '', '', '', '', '', '', nan],
 ['28', nan, '', '', nan, nan, nan, nan, '', '', '', '', '', '', nan],
 ['32', nan, '', '', nan, nan, nan, nan, '', '', '', '', '', '', nan]]

In [125]:
# Открываем существующий документ
doc = Document(row['file_path'].values[0])

# Находим первую таблицу в документе
table = doc.tables[0]

In [127]:
lfff =[]
for i in table.row_cells(7):
    lfff.append(i.text)

lfff

['8',
 '34',
 '',
 '',
 '',
 '14',
 '12',
 '11',
 '',
 '',
 '',
 '',
 '',
 '',
 '71',
 '71',
 '13',
 '11',
 '',
 '',
 '',
 'Е',
 '16',
 '13',
 '13,0',
 '5',
 '8',
 '',
 '']